In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '/data/ephemeral/home/Upstage_ML_Competition01/data/train.csv'
test_path  = '/data/ephemeral/home/Upstage_ML_Competition01/data/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train['is_test'] = 0
test['is_test'] = 1
concat = pd.concat([train, test])     # 하나의 데이터로 만들어줍니다.

In [ ]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

In [ ]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
concat['구'] = concat['시군구'].map(lambda x : x.split()[1])
concat['동'] = concat['시군구'].map(lambda x : x.split()[2])
del concat['시군구']

concat['계약년'] = concat['계약년월'].astype('str').map(lambda x : x[:4])
concat['계약월'] = concat['계약년월'].astype('str').map(lambda x : x[4:])

In [ ]:
# codebook 완성되면 바꿔야함
concat = concat.rename(columns={'전용면적(㎡)':'전용면적',
                                'k-단지분류(아파트,주상복합등등)' : '단지분류',
                                'k-전화번호' : '전화번호', 
                                'k-팩스번호' : '팩스번호',
                                'k-세대타입(분양형태)' : '분양형태',
                                'k-관리방식' : '관리방식',
                                'k-복도유형' : '복도유형',
                                'k-난방방식' : '난방방식',
                                'k-전체동수' : '전체동수',
                                'k-전체세대수' : '전체세대수',
                                'k-건설사(시공사)' : '시공사',
                                'k-시행사' : '시행사', 
                                'k-사용검사일-사용승인일' : '사용검사일-사용승인일',
                                'k-연면적' : '연면적',
                                'k-주거전용면적' : '주거전용면적',
                                'k-관리비부과면적' : '관리비부과면적',
                                'k-전용면적별세대현황(60㎡이하)' : '전용면적별세대현황(60㎡이하)',
                                'k-전용면적별세대현황(60㎡~85㎡이하)' : '전용면적별세대현황(60㎡~85㎡이하)',
                                'k-85㎡~135㎡이하' : '전용면적별세대현황(85㎡~135㎡이하)',
                                'k-135㎡초과' : '전용면적별세대현황(135㎡초과)',
                                'k-홈페이지' : '홈페이지',
                                'k-등록일자' : '등록일자',
                                'k-수정일자' : '수정일자',
                                '기타/의무/임대/임의=1/2/3/4' : '권리유형',
                                'target' : '집값(target)',
                                'is_test' : '테스트셋여부'
                                })

In [ ]:
# 사용할 변수 추출
using_list = ['구', '동','계약년월','계약년', '계약월', '아파트명', '전용면적', '계약일', '층', '건축년도',
       '도로명', '거래유형', '단지분류', '분양형태', '전체동수', '전체세대수', '시공사',
       '시행사', '연면적', '주거전용면적', '관리비부과면적', '전용면적별세대현황(60㎡이하)',
       '전용면적별세대현황(60㎡~85㎡이하)', '전용면적별세대현황(85㎡~135㎡이하)', '전용면적별세대현황(135㎡초과)',
       '건축면적', '주차대수', '권리유형', '단지신청일', '집값(target)', '테스트셋여부']

In [ ]:
using_concat = concat[using_list]
using_concat

In [ ]:
# 전체세대수에서 Nan인 값
using_concat[using_concat["테스트셋여부"]==0][["아파트명", "전체세대수", '전용면적별세대현황(60㎡이하)', '전용면적별세대현황(60㎡~85㎡이하)', '전용면적별세대현황(85㎡~135㎡이하)', '전용면적별세대현황(135㎡초과)', "집값(target)"]].head(20)

In [ ]:
using_concat[(using_concat["테스트셋여부"]==0) & (using_concat["전체세대수"].isna()==True)][["아파트명", "전체세대수", '전용면적별세대현황(60㎡이하)', '전용면적별세대현황(60㎡~85㎡이하)', '전용면적별세대현황(85㎡~135㎡이하)', '전용면적별세대현황(135㎡초과)', "집값(target)"]].head(30)

In [ ]:
len(using_concat[(using_concat["테스트셋여부"]==0) & (using_concat["전체세대수"].isna()==True)]["아파트명"].unique())